In [1]:
import gc
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

from subprocess import check_output
print(check_output(["ls", "input"]).decode("utf8"))

aisles.csv
chunk_0.pkl
departments.csv
order_products__prior.csv
order_products__train.csv
orders.csv
orders_comsum.pkl
previous_products.pkl
prod2vec.pkl
product_period.pkl
product_periods_stat.pkl
products.csv
sample_submission.csv
user_department_products.pkl



In [2]:
pwd

u'/Users/shaolongchen/Data Science/Kaggle/Instacart'

In [3]:
path_data = '/Users/shaolongchen/Data Science/Kaggle/Instacart/input/'

In [4]:
priors = pd.read_csv(path_data + 'order_products__prior.csv', 
                     dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})

In [5]:
priors.head(10)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
9,3,33754,1,1


In [6]:
train = pd.read_csv(path_data + 'order_products__train.csv', 
                    dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})

train.head(10)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
5,1,13176,6,0
6,1,47209,7,0
7,1,22035,8,1
8,36,39612,1,0
9,36,19660,2,1


In [7]:
orders = pd.read_csv(path_data + 'orders.csv', 
                         dtype={
                                'order_id': np.int32,
                                'user_id': np.int64,
                                'eval_set': 'category',
                                'order_number': np.int16,
                                'order_dow': np.int8,
                                'order_hour_of_day': np.int8,
                                'days_since_prior_order': np.float32})

In [8]:
orders.head(20)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [9]:
orders.groupby("eval_set")["user_id"].agg(lambda x:len(np.unique(x)))

eval_set
prior    206209
test      75000
train    131209
Name: user_id, dtype: int64

In [10]:
products = pd.read_csv(path_data + 'products.csv')
aisles = pd.read_csv(path_data + "aisles.csv")
departments = pd.read_csv(path_data + "departments.csv")
sample_submission = pd.read_csv(path_data + "sample_submission.csv")

In [11]:
class tick_tock:
    def __init__(self, process_name, verbose=1):
        self.process_name = process_name
        self.verbose = verbose
    def __enter__(self):
        if self.verbose:
            print(self.process_name + " begin ......")
            self.begin_time = time.time()
    def __exit__(self, type, value, traceback):
        if self.verbose:
            end_time = time.time()
            print(self.process_name + " end ......")
            print('time lapsing {0} s \n'.format(end_time - self.begin_time))

In [12]:
def ka_add_groupby_features_1_vs_n(df, group_columns_list, agg_dict, only_new_feature=True):
    '''Create statistical columns, group by [N columns] and compute stats on [N column]

       Parameters
       ----------
       df: pandas dataframe
          Features matrix
       group_columns_list: list_like
          List of columns you want to group with, could be multiple columns
       agg_dict: python dictionary

       Return
       ------
       new pandas dataframe with original columns and new added columns

       Example
       -------
       {real_column_name: {your_specified_new_column_name : method}}
       agg_dict = {'user_id':{'prod_tot_cnts':'count'},
                   'reordered':{'reorder_tot_cnts_of_this_prod':'sum'},
                   'user_buy_product_times': {'prod_order_once':lambda x: sum(x==1),
                                              'prod_order_more_than_once':lambda x: sum(x==2)}}
       ka_add_stats_features_1_vs_n(train, ['product_id'], agg_dict)
    '''
    with tick_tock("add stats features"):
        try:
            if type(group_columns_list) == list:
                pass
            else:
                raise TypeError(k + "should be a list")
        except TypeError as e:
            print(e)
            raise

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped.agg(agg_dict)
        the_stats.columns = the_stats.columns.droplevel(0)
        the_stats.reset_index(inplace=True)
        if only_new_feature:
            df_new = the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')

    return df_new

In [13]:
priors_orders_detail = orders.merge(right=priors, how='inner', on='order_id')

In [14]:
priors_orders_detail.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0
5,2398795,1,prior,2,3,7,15.0,196,1,1
6,2398795,1,prior,2,3,7,15.0,10258,2,0
7,2398795,1,prior,2,3,7,15.0,12427,3,1
8,2398795,1,prior,2,3,7,15.0,13176,4,0
9,2398795,1,prior,2,3,7,15.0,26088,5,1


In [15]:
priors_orders_detail.loc[:,'_user_buy_product_times'] = priors_orders_detail.groupby(['user_id', 'product_id']).cumcount() + 1

In [16]:
priors_orders_detail.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_user_buy_product_times
0,2539329,1,prior,1,2,8,NaN,196,1,0,1
1,2539329,1,prior,1,2,8,NaN,14084,2,0,1
2,2539329,1,prior,1,2,8,NaN,12427,3,0,1
3,2539329,1,prior,1,2,8,NaN,26088,4,0,1
4,2539329,1,prior,1,2,8,NaN,26405,5,0,1
5,2398795,1,prior,2,3,7,15.0,196,1,1,2
6,2398795,1,prior,2,3,7,15.0,10258,2,0,1
7,2398795,1,prior,2,3,7,15.0,12427,3,1,2
8,2398795,1,prior,2,3,7,15.0,13176,4,0,1
9,2398795,1,prior,2,3,7,15.0,26088,5,1,2


In [17]:
agg_dict = {'user_id':{'_prod_tot_cnts':'count'}, 
            'reordered':{'_prod_reorder_tot_cnts':'sum'}, 
            '_user_buy_product_times': {'_prod_buy_first_time_total_cnt':lambda x: sum(x==1),
                                        '_prod_buy_second_time_total_cnt':lambda x: sum(x==2)}}

In [18]:
prd = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['product_id'], agg_dict)

add stats features begin ......


/Users/shaolongchen/anaconda/lib/python2.7/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


add stats features end ......
time lapsing 223.572649002 s 



In [19]:
df = priors_orders_detail
group_columns_list = df.product_id

In [20]:
df_new = df.copy()
grouped = df_new.groupby(group_columns_list)
grouped

In [21]:
grouped.agg({'user_id':{'_prod_tot_cnts':'count'}}).head()

,user_id
,_prod_tot_cnts
product_id,
1,1852
2,90
3,277
4,329
5,15


In [22]:
grouped.agg({'reordered':{'_prod_reorder_tot_cnts':'sum'}}).head()

,reordered
,_prod_reorder_tot_cnts
product_id,
1,1136.0
2,12.0
3,203.0
4,147.0
5,9.0


In [23]:
grouped.agg({'_user_buy_product_times': {'_prod_buy_first_time_total_cnt':lambda x: sum(x==1)}}).head()

,_user_buy_product_times
,_prod_buy_first_time_total_cnt
product_id,
1,716
2,78
3,74
4,182
5,6


In [24]:
grouped.agg({'_user_buy_product_times': {'_prod_buy_first_time_total_cnt':lambda x: sum(x==1),
                                        '_prod_buy_second_time_total_cnt':lambda x: sum(x==2)}}).head()

_user_buy_product_times                               
           _prod_buy_second_time_total_cnt _prod_buy_first_time_total_cnt
product_id                                                               
1                                      276                            716
2                                        8                             78
3                                       36                             74
4                                       64                            182
5                                        4                              6

In [25]:
orders.groupby("eval_set")["user_id"].aggregate(lambda x:len(np.unique(x)))

eval_set
prior    206209
test      75000
train    131209
Name: user_id, dtype: int64

In [26]:
the_stats = grouped.agg(agg_dict)

In [27]:
the_stats.head(10)

_user_buy_product_times                                 \
           _prod_buy_second_time_total_cnt _prod_buy_first_time_total_cnt   
product_id                                                                  
1                                      276                            716   
2                                        8                             78   
3                                       36                             74   
4                                       64                            182   
5                                        4                              6   
6                                        2                              5   
7                                        6                             18   
8                                       30                             82   
9                                       31                             74   
10                                     399                           1268   

                  user_id              reordered  
           _prod_tot_cnts _prod_reorder_tot_cnts  
product_id                                        
1                    1852                 1136.0  
2                      90                   12.0  
3                     277                  203.0  
4                     329                  147.0  
5                      15                    9.0  
6                       8                    3.0  
7                      30                   12.0  
8                     165                   83.0  
9                     156                   82.0  
10                   2572                 1304.0

In [28]:
the_stats.columns

MultiIndex(levels=[[u'_user_buy_product_times', u'user_id', u'reordered'], [u'_prod_buy_first_time_total_cnt', u'_prod_buy_second_time_total_cnt', u'_prod_reorder_tot_cnts', u'_prod_tot_cnts']],
           labels=[[0, 0, 1, 2], [1, 0, 3, 2]])

In [29]:
the_stats.columns = the_stats.columns.droplevel(0)
the_stats.columns

Index([u'_prod_buy_second_time_total_cnt', u'_prod_buy_first_time_total_cnt',
       u'_prod_tot_cnts', u'_prod_reorder_tot_cnts'],
      dtype='object')

In [30]:
the_stats.reset_index(inplace=True)

In [31]:
the_stats.head()

,product_id,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts
0,1,276,716,1852,1136.0
1,2,8,78,90,12.0
2,3,36,74,277,203.0
3,4,64,182,329,147.0
4,5,4,6,15,9.0


In [32]:
prd = the_stats
prd.head()

,product_id,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts
0,1,276,716,1852,1136.0
1,2,8,78,90,12.0
2,3,36,74,277,203.0
3,4,64,182,329,147.0
4,5,4,6,15,9.0


In [33]:
prd['_prod_reorder_prob'] = prd._prod_buy_second_time_total_cnt / prd._prod_buy_first_time_total_cnt
prd['_prod_reorder_ratio'] = prd._prod_reorder_tot_cnts / prd._prod_tot_cnts
prd['_prod_reorder_times'] = 1 + prd._prod_reorder_tot_cnts / prd._prod_buy_first_time_total_cnt

In [34]:
prd.head()

,product_id,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,_prod_reorder_prob,_prod_reorder_ratio,_prod_reorder_times
0,1,276,716,1852,1136.0,0.385475,0.613391,2.586592
1,2,8,78,90,12.0,0.102564,0.133333,1.153846
2,3,36,74,277,203.0,0.486486,0.732852,3.743243
3,4,64,182,329,147.0,0.351648,0.446809,1.807692
4,5,4,6,15,9.0,0.666667,0.600000,2.500000


In [35]:
agg_dict_2 = {'order_number':{'_user_total_orders':'max'},
              'days_since_prior_order':{'_user_sum_days_since_prior_order':'sum', 
                                        '_user_mean_days_since_prior_order': 'mean'}}

In [36]:
orders[orders.eval_set == 'prior'].head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [37]:
group_columns_list2 = orders.user_id

In [38]:
df2 = orders[orders.eval_set == 'prior']

In [39]:
df_new2 = df2.copy()

In [40]:
grouped2 = df_new2.groupby(group_columns_list2)

In [41]:
grouped2

In [42]:
grouped2.agg({'order_number':{'_user_total_orders':'max'}}).head()

,order_number
,_user_total_orders
user_id,
1,10
2,14
3,12
4,5
5,4


In [43]:
grouped2.agg({'days_since_prior_order':{'_user_sum_days_since_prior_order':'sum'}}).head()

,days_since_prior_order
,_user_sum_days_since_prior_order
user_id,
1,176.0
2,198.0
3,133.0
4,55.0
5,40.0


In [44]:
grouped2.agg({'days_since_prior_order':{'_user_mean_days_since_prior_order': 'mean'}}).head()

,days_since_prior_order
,_user_mean_days_since_prior_order
user_id,
1,19.555555
2,15.230769
3,12.090909
4,13.750000
5,13.333333


In [45]:
the_stats2 = grouped2.agg(agg_dict_2)

In [46]:
the_stats2.head()

days_since_prior_order                                   \
        _user_mean_days_since_prior_order _user_sum_days_since_prior_order   
user_id                                                                      
1                               19.555555                            176.0   
2                               15.230769                            198.0   
3                               12.090909                            133.0   
4                               13.750000                             55.0   
5                               13.333333                             40.0   

              order_number  
        _user_total_orders  
user_id                     
1                       10  
2                       14  
3                       12  
4                        5  
5                        4

In [47]:
the_stats2.columns = the_stats2.columns.droplevel(0)
the_stats2.reset_index(inplace=True)

In [48]:
the_stats2.head()

,user_id,_user_mean_days_since_prior_order,_user_sum_days_since_prior_order,_user_total_orders
0,1,19.555555,176.0,10
1,2,15.230769,198.0,14
2,3,12.090909,133.0,12
3,4,13.750000,55.0,5
4,5,13.333333,40.0,4


In [49]:
users = ka_add_groupby_features_1_vs_n(orders[orders.eval_set == 'prior'], ['user_id'], agg_dict_2)

add stats features begin ......
add stats features end ......
time lapsing 0.69935297966 s 



In [50]:
users.head()

,user_id,_user_mean_days_since_prior_order,_user_sum_days_since_prior_order,_user_total_orders
0,1,19.555555,176.0,10
1,2,15.230769,198.0,14
2,3,12.090909,133.0,12
3,4,13.750000,55.0,5
4,5,13.333333,40.0,4


In [51]:
priors_orders_detail.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_user_buy_product_times
0,2539329,1,prior,1,2,8,NaN,196,1,0,1
1,2539329,1,prior,1,2,8,NaN,14084,2,0,1
2,2539329,1,prior,1,2,8,NaN,12427,3,0,1
3,2539329,1,prior,1,2,8,NaN,26088,4,0,1
4,2539329,1,prior,1,2,8,NaN,26405,5,0,1


In [52]:
group_columns_list3 = priors_orders_detail.user_id

In [53]:
df_new3 = priors_orders_detail.copy()
grouped3 = df_new3.groupby(group_columns_list3)

In [54]:
agg_dict_3 = {'reordered':
              {'_user_reorder_ratio': 
               lambda x: float(sum(priors_orders_detail.ix[x.index,'reordered']==1))/
                         sum(priors_orders_detail.ix[x.index,'order_number'] > 1)},
              'product_id':{'_user_total_products':'count', 
                            '_user_distinct_products': lambda x: x.nunique()}}

In [55]:
grouped3.agg(agg_dict_3).head()

/Users/shaolongchen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


product_id                                   reordered
        _user_total_products _user_distinct_products _user_reorder_ratio
user_id                                                                 
1                         59                      18            0.759259
2                        195                     102            0.510989
3                         88                      33            0.705128
4                         18                      17            0.071429
5                         37                      23            0.538462

In [56]:
the_stats3 = grouped3.agg(agg_dict_3)
the_stats3.columns = the_stats3.columns.droplevel(0)
the_stats3.reset_index(inplace=True)
print "processing is done"

processing is done


In [57]:
the_stats3.head(20)

,user_id,_user_total_products,_user_distinct_products,_user_reorder_ratio
0,1,59,18,0.759259
1,2,195,102,0.510989
2,3,88,33,0.705128
3,4,18,17,0.071429
4,5,37,23,0.538462
5,6,14,12,0.200000
6,7,206,68,0.711340
7,8,49,36,0.464286
8,9,76,58,0.391304
9,10,143,94,0.355072


In [58]:
us = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['user_id'], agg_dict_3)

add stats features begin ......
add stats features end ......
time lapsing 1533.67823005 s 



In [59]:
us.head()

,user_id,_user_total_products,_user_distinct_products,_user_reorder_ratio
0,1,59,18,0.759259
1,2,195,102,0.510989
2,3,88,33,0.705128
3,4,18,17,0.071429
4,5,37,23,0.538462


In [60]:
users = users.merge(us, how='inner')

In [61]:
users['_user_average_basket'] = users._user_total_products / users._user_total_orders

us = orders[orders.eval_set != "prior"][['user_id', 'order_id', 'eval_set', 'days_since_prior_order']]
us.rename(index=str, columns={'days_since_prior_order': 'time_since_last_order'}, inplace=True)

In [62]:
users = users.merge(us, how='inner')
users.head()

,user_id,_user_mean_days_since_prior_order,_user_sum_days_since_prior_order,_user_total_orders,_user_total_products,_user_distinct_products,_user_reorder_ratio,_user_average_basket,order_id,eval_set,time_since_last_order
0,1,19.555555,176.0,10,59,18,0.759259,5.900000,1187899,train,14.0
1,2,15.230769,198.0,14,195,102,0.510989,13.928571,1492625,train,30.0
2,3,12.090909,133.0,12,88,33,0.705128,7.333333,2774568,test,11.0
3,4,13.750000,55.0,5,18,17,0.071429,3.600000,329954,test,30.0
4,5,13.333333,40.0,4,37,23,0.538462,9.250000,2196797,train,6.0


In [63]:
agg_dict_4 = {'order_number':{'_up_order_count': 'count', 
                              '_up_first_order_number': 'min', 
                              '_up_last_order_number':'max'}, 
              'add_to_cart_order':{'_up_average_cart_position': 'mean'}}

In [64]:
data = ka_add_groupby_features_1_vs_n(df=priors_orders_detail, 
                                                      group_columns_list=['user_id', 'product_id'], 
                                                      agg_dict=agg_dict_4)
data.head()

add stats features begin ......
add stats features end ......
time lapsing 33.330685854 s 



,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number
0,1,196,1.400000,10,1,10
1,1,10258,3.333333,9,2,10
2,1,10326,5.000000,1,5,5
3,1,12427,3.300000,10,1,10
4,1,13032,6.333333,3,2,10


In [65]:
data = data.merge(prd, how='inner', on='product_id').merge(users, how='inner', on='user_id')
data.head()

,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,...,_user_mean_days_since_prior_order,_user_sum_days_since_prior_order,_user_total_orders,_user_total_products,_user_distinct_products,_user_reorder_ratio,_user_average_basket,order_id,eval_set,time_since_last_order
0,1,196,1.400000,10,1,10,4660,8000,35791,27791.0,...,19.555555,176.0,10,59,18,0.759259,5.9,1187899,train,14.0
1,1,10258,3.333333,9,2,10,308,557,1946,1389.0,...,19.555555,176.0,10,59,18,0.759259,5.9,1187899,train,14.0
2,1,10326,5.000000,1,5,5,1003,1923,5526,3603.0,...,19.555555,176.0,10,59,18,0.759259,5.9,1187899,train,14.0
3,1,12427,3.300000,10,1,10,889,1679,6476,4797.0,...,19.555555,176.0,10,59,18,0.759259,5.9,1187899,train,14.0
4,1,13032,6.333333,3,2,10,617,1286,3751,2465.0,...,19.555555,176.0,10,59,18,0.759259,5.9,1187899,train,14.0


In [66]:
data['_up_order_rate'] = data._up_order_count / data._user_total_orders
data['_up_order_since_last_order'] = data._user_total_orders - data._up_last_order_number
data['_up_order_rate_since_first_order'] = data._up_order_count / (data._user_total_orders - data._up_first_order_number + 1)

In [67]:
train = train.merge(right=orders[['order_id', 'user_id']], how='left', on='order_id')
train.head()

,order_id,product_id,add_to_cart_order,reordered,user_id
0,1,49302,1,1,112108
1,1,11109,2,1,112108
2,1,10246,3,0,112108
3,1,49683,4,0,112108
4,1,43633,5,1,112108


In [68]:
data = data.merge(train[['user_id', 'product_id', 'reordered']], on=['user_id', 'product_id'], how='left')
data.head()

,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,...,_user_distinct_products,_user_reorder_ratio,_user_average_basket,order_id,eval_set,time_since_last_order,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order,reordered
0,1,196,1.400000,10,1,10,4660,8000,35791,27791.0,...,18,0.759259,5.9,1187899,train,14.0,1.0,0,1.000000,1.0
1,1,10258,3.333333,9,2,10,308,557,1946,1389.0,...,18,0.759259,5.9,1187899,train,14.0,0.9,0,1.000000,1.0
2,1,10326,5.000000,1,5,5,1003,1923,5526,3603.0,...,18,0.759259,5.9,1187899,train,14.0,0.1,5,0.166667,NaN
3,1,12427,3.300000,10,1,10,889,1679,6476,4797.0,...,18,0.759259,5.9,1187899,train,14.0,1.0,0,1.000000,NaN
4,1,13032,6.333333,3,2,10,617,1286,3751,2465.0,...,18,0.759259,5.9,1187899,train,14.0,0.3,0,0.333333,1.0


In [69]:
del priors_orders_detail, orders
gc.collect()

913

In [70]:
import xgboost

/Users/shaolongchen/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [71]:
train = data.loc[data.eval_set == "train",:]

In [72]:
train.drop(['eval_set', 'user_id', 'product_id', 'order_id'], axis=1, inplace=True)
train.head()

/Users/shaolongchen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,_prod_reorder_prob,_prod_reorder_ratio,...,_user_total_orders,_user_total_products,_user_distinct_products,_user_reorder_ratio,_user_average_basket,time_since_last_order,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order,reordered
0,1.400000,10,1,10,4660,8000,35791,27791.0,0.582500,0.776480,...,10,59,18,0.759259,5.9,14.0,1.0,0,1.000000,1.0
1,3.333333,9,2,10,308,557,1946,1389.0,0.552962,0.713772,...,10,59,18,0.759259,5.9,14.0,0.9,0,1.000000,1.0
2,5.000000,1,5,5,1003,1923,5526,3603.0,0.521581,0.652009,...,10,59,18,0.759259,5.9,14.0,0.1,5,0.166667,NaN
3,3.300000,10,1,10,889,1679,6476,4797.0,0.529482,0.740735,...,10,59,18,0.759259,5.9,14.0,1.0,0,1.000000,NaN
4,6.333333,3,2,10,617,1286,3751,2465.0,0.479782,0.657158,...,10,59,18,0.759259,5.9,14.0,0.3,0,0.333333,1.0


In [73]:
train.loc[:, 'reordered'] = train.reordered.fillna(0)
train.head()

/Users/shaolongchen/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,_prod_reorder_prob,_prod_reorder_ratio,...,_user_total_orders,_user_total_products,_user_distinct_products,_user_reorder_ratio,_user_average_basket,time_since_last_order,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order,reordered
0,1.400000,10,1,10,4660,8000,35791,27791.0,0.582500,0.776480,...,10,59,18,0.759259,5.9,14.0,1.0,0,1.000000,1.0
1,3.333333,9,2,10,308,557,1946,1389.0,0.552962,0.713772,...,10,59,18,0.759259,5.9,14.0,0.9,0,1.000000,1.0
2,5.000000,1,5,5,1003,1923,5526,3603.0,0.521581,0.652009,...,10,59,18,0.759259,5.9,14.0,0.1,5,0.166667,0.0
3,3.300000,10,1,10,889,1679,6476,4797.0,0.529482,0.740735,...,10,59,18,0.759259,5.9,14.0,1.0,0,1.000000,0.0
4,6.333333,3,2,10,617,1286,3751,2465.0,0.479782,0.657158,...,10,59,18,0.759259,5.9,14.0,0.3,0,0.333333,1.0


In [74]:
X_test = data.loc[data.eval_set == "test",:]
X_test.head()

,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,...,_user_distinct_products,_user_reorder_ratio,_user_average_basket,order_id,eval_set,time_since_last_order,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order,reordered
18,15,196,2.200,5,15,22,4660,8000,35791,27791.0,...,13,0.867647,3.272727,2161313,test,7.0,0.227273,0,0.625000,NaN
19,15,12427,2.100,10,1,20,889,1679,6476,4797.0,...,13,0.867647,3.272727,2161313,test,7.0,0.454545,2,0.454545,NaN
20,15,1747,3.500,4,8,19,221,562,1448,886.0,...,13,0.867647,3.272727,2161313,test,7.0,0.181818,3,0.266667,NaN
21,15,10441,2.375,8,1,22,465,867,2909,2042.0,...,13,0.867647,3.272727,2161313,test,7.0,0.363636,0,0.363636,NaN
22,15,11266,1.600,10,1,19,645,1081,4081,3000.0,...,13,0.867647,3.272727,2161313,test,7.0,0.454545,3,0.454545,NaN


In [75]:
X_train, X_val, y_train, y_val = train_test_split(train.drop('reordered', axis=1), train.reordered,
                                                    test_size=0.9, random_state=42)

In [76]:
d_train = xgboost.DMatrix(X_train, y_train)
xgb_params = {
    "objective"         : "reg:logistic"
    ,"eval_metric"      : "logloss"
    ,"eta"              : 0.1
    ,"max_depth"        : 6
    ,"min_child_weight" :10
    ,"gamma"            :0.70
    ,"subsample"        :0.76
    ,"colsample_bytree" :0.95
    ,"alpha"            :2e-05
    ,"lambda"           :10
}

In [ ]:
watchlist= [(d_train, "train")]
watchlist

[(<xgboost.core.DMatrix at 0x113365c50>, 'train')]

In [ ]:
bst = xgboost.train(params=xgb_params, dtrain=d_train, num_boost_round=80, evals=watchlist, verbose_eval=10)
bst

[0]	train-logloss:0.625713


In [ ]:
xgboost.plot_importance(bst)

In [ ]:
d_test = xgboost.DMatrix(X_test.drop(['eval_set', 'user_id', 'order_id', 'reordered', 'product_id'], axis=1))
d_test

In [ ]:
X_test.loc[:,'reordered'] = (bst.predict(d_test) > 0.2).astype(int)
X_test

In [ ]:
X_test.loc[:, 'product_id'] = X_test.product_id.astype(str)
X_test

In [ ]:
def ka_add_groupby_features_n_vs_1(df, group_columns_list, target_columns_list, methods_list, keep_only_stats=True, verbose=1):
    '''Create statistical columns, group by [N columns] and compute stats on [1 column]

       Parameters
       ----------
       df: pandas dataframe
          Features matrix
       group_columns_list: list_like
          List of columns you want to group with, could be multiple columns
       target_columns_list: list_like
          column you want to compute stats, need to be a list with only one element
       methods_list: list_like
          methods that you want to use, all methods that supported by groupby in Pandas

       Return
       ------
       new pandas dataframe with original columns and new added columns

       Example
       -------
       ka_add_stats_features_n_vs_1(train, group_columns_list=['x0'], target_columns_list=['x10'])
    '''
    with tick_tock("add stats features", verbose):
        dicts = {"group_columns_list": group_columns_list , "target_columns_list": target_columns_list, "methods_list" :methods_list}

        for k, v in dicts.items():
            try:
                if type(v) == list:
                    pass
                else:
                    raise TypeError(k + "should be a list")
            except TypeError as e:
                print(e)
                raise

        grouped_name = ''.join(group_columns_list)
        target_name = ''.join(target_columns_list)
        combine_name = [[grouped_name] + [method_name] + [target_name] for method_name in methods_list]

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped[target_name].agg(methods_list).reset_index()
        the_stats.columns = [grouped_name] + \
                            ['_%s_%s_by_%s' % (grouped_name, method_name, target_name) \
                             for (grouped_name, method_name, target_name) in combine_name]
        if keep_only_stats:
            return the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')
        return df_new

In [ ]:
df_new4 = X_test[X_test.reordered == 1].copy()

In [ ]:
group_columns_list = df_new4.order_id
target_columns_list = df_new4.product_id
methods_list=[lambda x: ' '.join(set(x))]

In [ ]:
grouped_name = ''.join(group_columns_list)
print grouped_name
target_name = ''.join(target_columns_list)
print target_name
combine_name = [[grouped_name] + [method_name] + [target_name] for method_name in methods_list]
print combine_name

In [ ]:
the_stats = df_new4.groupby(group_columns_list)[target_name].agg(methods_list).reset_index()

In [ ]:
the_stats.columns = [grouped_name] + \
                            ['_%s_%s_by_%s' % (grouped_name, method_name, target_name) \
                             for (grouped_name, method_name, target_name) in combine_name]

In [ ]:
the_stats.head()

In [ ]:
submit = ka_add_groupby_features_n_vs_1(X_test[X_test.reordered == 1], 
                                               group_columns_list=['order_id'],
                                               target_columns_list= ['product_id'],
                                               methods_list=[lambda x: ' '.join(set(x))], keep_only_stats=True)
submit.columns = sample_submission.columns.tolist()
submit_final = sample_submission[['order_id']].merge(submit, how='left').fillna('None')
submit_final.to_csv("08-09-1.csv", index=False)